In [ ]:
from IPython.display import display
import os
import shutil

In [ ]:
import numpy as np
import glob
import numpy as np
import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
IMAGE_SIZE = 400
BATCH_SIZE = 8
CHANNELS = 3
EPOCHS = 1

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "./initial training set", 
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
train_size = 0.8
train_set_length = len(dataset) * train_size
train_set_length

In [ ]:
train_ds= dataset.take(16)

In [ ]:
len(train_ds)

In [ ]:
test_ds = dataset.skip(16) 
len(test_ds)

In [ ]:
val_size = 0.1
len(dataset)*val_size

In [ ]:
val_ds = test_ds.take(2)

In [ ]:
len(val_ds)

In [ ]:
test_ds = test_ds.skip(2)
len(test_ds)

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split= 0.1, test_split = 0.1, shuffle=True, shuffle_size =10000):
    ds_size = len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
len(train_ds)
len(val_ds)
len(test_ds)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras import models, layers

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2) 
])

In [ ]:
Input_shape=(BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE, CHANNELS)
n_classes = 2
model = models.Sequential([
resize_and_rescale, 
                          data_augmentation,
                          layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE,IMAGE_SIZE)),
                          layers.MaxPooling2D((2,2)),
                          ##########################
                          layers.Conv2D(64, kernel_size = (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, kernel_size = (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Flatten(),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(n_classes, activation='softmax')
])
model.build(input_shape=Input_shape)

In [ ]:
model.summary()
                        

In [ ]:
model.compile(
    optimizer ='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

In [ ]:
# history = model.fit(
#     train_ds,
#     epochs = EPOCHS,
#     batch_size = BATCH_SIZE,
#     verbose=1,
#     validation_data=val_ds
    
# )

In [ ]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10))


# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# trainImages = np.asarray(trainImages)
# trainLabels = np.asarray(trainLabels).astype('float32')
# validationLabels = np.asarray(validationLabels).astype('float32')
# validationImages = np.asarray(validationImages)

# history = model.fit(trainImages, trainLabels, epochs=10, 
#                     validation_data=(validationImages, validationLabels))

In [ ]:
# trainImages.shape()
# trainLabels.shape()

# validationImages.shape()
# validationLabels.shape()

In [ ]:
# (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
